In [3]:
import numpy as np
import keras
from keras.models import Model
from keras.layers import Input, SimpleRNN, Dense, Lambda, LSTM
from keras import backend as K

Using TensorFlow backend.


How many charactars in the our data?

In [4]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


In [5]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
char_to_ix["\n"] = 0
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
ix_to_char[27] = ""
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 27: ''}


Let's create our training set:

In [6]:
# open the file again, and read the lines
with open("dinos.txt") as f:
    examples = f.readlines()
examples = [x.lower().strip() for x in examples]
# Shuffle list of all dinosaur names
np.random.seed(210)
np.random.shuffle(examples)

In [7]:
# compute the max length of a word in data
maxLen = len(max(examples, key=len))
print(maxLen)

26


In [8]:
m = len(examples)
feature_size = 1  # we use the actual index, and not 1-hot-encoding

X = np.full((m, maxLen, feature_size), 27)
Y = np.full((m, maxLen, feature_size), 27)

for i in range(m):
    word = examples[i]
    l = len(word)        
    X[i, 0:l, :] = np.array([char_to_ix[ch] for ch in word]).reshape(-1, 1)
    Y[i, 0:l, :] = np.concatenate([X[i, 1:l, :], [[char_to_ix["\n"]]]])

In [9]:
# create the model
def simple_model(input_shape, n_a = 50):
    inp = Input(shape=input_shape, name='x')
    X = SimpleRNN(n_a, return_sequences=True)(inp)
    X = Dense(vocab_size+1, activation='softmax')(X)
    model = Model(inputs=inp, outputs=X)
    return model

In [10]:
model = simple_model(input_shape=(maxLen, feature_size))
# Use sparse loss due to not using 1-hot-encoding
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
model.fit(X, Y, epochs = 100, batch_size = 32, shuffle=True)

Epoch 1/100
1536/1536 [==============================] - 1s 920us/step - loss: 2.4010 - sparse_categorical_accuracy: 0.3905
Epoch 2/100
1536/1536 [==============================] - 1s 525us/step - loss: 1.6093 - sparse_categorical_accuracy: 0.5642
Epoch 3/100
1536/1536 [==============================] - 1s 540us/step - loss: 1.4377 - sparse_categorical_accuracy: 0.5810
Epoch 4/100
1536/1536 [==============================] - 1s 530us/step - loss: 1.3422 - sparse_categorical_accuracy: 0.6346
Epoch 5/100
1536/1536 [==============================] - 1s 529us/step - loss: 1.2661 - sparse_categorical_accuracy: 0.6652
Epoch 6/100
1536/1536 [==============================] - 1s 533us/step - loss: 1.2063 - sparse_categorical_accuracy: 0.6808
Epoch 7/100
1536/1536 [==============================] - 1s 533us/step - loss: 1.1585 - sparse_categorical_accuracy: 0.6835
Epoch 8/100
1536/1536 [==============================] - 1s 567us/step - loss: 1.1208 - sparse_categorical_accuracy: 0.6860
Epoch 9/

1536/1536 [==============================] - 1s 534us/step - loss: 0.9084 - sparse_categorical_accuracy: 0.7338
Epoch 66/100
1536/1536 [==============================] - 1s 557us/step - loss: 0.9065 - sparse_categorical_accuracy: 0.7349
Epoch 67/100
1536/1536 [==============================] - 1s 555us/step - loss: 0.9054 - sparse_categorical_accuracy: 0.7356
Epoch 68/100
1536/1536 [==============================] - 1s 552us/step - loss: 0.9044 - sparse_categorical_accuracy: 0.7356
Epoch 69/100
1536/1536 [==============================] - 1s 541us/step - loss: 0.9023 - sparse_categorical_accuracy: 0.7366
Epoch 70/100
1536/1536 [==============================] - 1s 553us/step - loss: 0.9014 - sparse_categorical_accuracy: 0.7363
Epoch 71/100
1536/1536 [==============================] - 1s 547us/step - loss: 0.9012 - sparse_categorical_accuracy: 0.7374
Epoch 72/100
1536/1536 [==============================] - 1s 530us/step - loss: 0.8998 - sparse_categorical_accuracy: 0.7378
Epoch 73/100


We got 75% accuracy after 100-200 epochs. Let's try LSTM

In [11]:
# create the model
def lstm_model(input_shape, n_a = 50):
    inp = Input(shape=input_shape, name='x')
    X = LSTM(n_a, return_sequences=True)(inp)
    X = Dense(vocab_size+1, activation='softmax')(X)
    model = Model(inputs=inp, outputs=X)
    return model

In [12]:
lstm = lstm_model(input_shape=(maxLen, feature_size))
lstm.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])
lstm.fit(X, Y, epochs = 100, batch_size = 32, shuffle=True)

Epoch 1/100
1536/1536 [==============================] - 2s 2ms/step - loss: 2.7497 - sparse_categorical_accuracy: 0.3500
Epoch 2/100
1536/1536 [==============================] - 2s 1ms/step - loss: 1.6225 - sparse_categorical_accuracy: 0.5701
Epoch 3/100
1536/1536 [==============================] - 2s 1ms/step - loss: 1.3245 - sparse_categorical_accuracy: 0.6016
Epoch 4/100
1536/1536 [==============================] - 2s 1ms/step - loss: 1.2386 - sparse_categorical_accuracy: 0.6386
Epoch 5/100
1536/1536 [==============================] - 2s 1ms/step - loss: 1.1793 - sparse_categorical_accuracy: 0.6782
Epoch 6/100
1536/1536 [==============================] - 2s 1ms/step - loss: 1.1327 - sparse_categorical_accuracy: 0.6937
Epoch 7/100
1536/1536 [==============================] - 2s 1ms/step - loss: 1.0987 - sparse_categorical_accuracy: 0.6990
Epoch 8/100
1536/1536 [==============================] - 2s 1ms/step - loss: 1.0753 - sparse_categorical_accuracy: 0.7001
Epoch 9/100
1536/1536 [=

Epoch 67/100
1536/1536 [==============================] - 2s 1ms/step - loss: 0.8758 - sparse_categorical_accuracy: 0.7450
Epoch 68/100
1536/1536 [==============================] - 2s 1ms/step - loss: 0.8750 - sparse_categorical_accuracy: 0.7469
Epoch 69/100
1536/1536 [==============================] - 2s 1ms/step - loss: 0.8735 - sparse_categorical_accuracy: 0.7450
Epoch 70/100
1536/1536 [==============================] - 2s 1ms/step - loss: 0.8722 - sparse_categorical_accuracy: 0.7459
Epoch 71/100
1536/1536 [==============================] - 2s 1ms/step - loss: 0.8706 - sparse_categorical_accuracy: 0.7460
Epoch 72/100
1536/1536 [==============================] - 2s 1ms/step - loss: 0.8694 - sparse_categorical_accuracy: 0.7466
Epoch 73/100
1536/1536 [==============================] - 2s 1ms/step - loss: 0.8677 - sparse_categorical_accuracy: 0.7473
Epoch 74/100
1536/1536 [==============================] - 2s 1ms/step - loss: 0.8663 - sparse_categorical_accuracy: 0.7473
Epoch 75/100
153

In [21]:
print(lstm.output_shape)

(None, 26, 28)


In [25]:
len([0])

26

In [26]:
pred = lstm.predict(X)

In [34]:
preds_ch = []
for p in preds:
    w = []
    for i in p:
        w.append(ix_to_char[i])
    preds_ch.append(w)

In [42]:
preds_ch[15]

['a',
 'n',
 'a',
 'a',
 'n',
 'o',
 's',
 's',
 'o',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [43]:
examples[15]

'qiupalong'